In [1]:
# Advent of Code 2023
# Day 25 Problem 1
import re
from collections import Counter
import functools

with open("aoc_25_input.txt") as f:
    A = f.read().strip().split("\n")

# c+p test case here
TEST = """jqt: rhn xhk nvd
rsh: frs pzl lsr
xhk: hfx
cmg: qnr nvd lhk bvb
rhn: xhk bvb hfx
bvb: xhk hfx
pzl: lsr hfx nvd
qnr: nvd
ntq: jqt hfx bvb xhk
nvd: lhk
lsr: lhk
rzs: qnr cmg lsr rsh
frs: qnr lhk lsr""".strip().split("\n")

for i, line in enumerate(TEST):
    print(f"{i}:\t{line}")

0:	jqt: rhn xhk nvd
1:	rsh: frs pzl lsr
2:	xhk: hfx
3:	cmg: qnr nvd lhk bvb
4:	rhn: xhk bvb hfx
5:	bvb: xhk hfx
6:	pzl: lsr hfx nvd
7:	qnr: nvd
8:	ntq: jqt hfx bvb xhk
9:	nvd: lhk
10:	lsr: lhk
11:	rzs: qnr cmg lsr rsh
12:	frs: qnr lhk lsr


In [2]:
from collections import defaultdict
class Graph:
    def __init__(self, inp) -> None:
        self.nodes = dict()
        self.edges = []
        for line in inp:
            left, right = line.split(": ")
            left = left.strip()

            if left not in self.nodes.keys():
                self.nodes[left] = Node(left)

            for rr in [r.strip() for r in right.split(" ")]:
                if rr not in self.nodes.keys():
                    self.nodes[rr] = Node(rr)
                #print(f"Adding edge between {left} and {rr}")
                e = Edge(self.nodes[left], self.nodes[rr])
                self.edges.append(e)

class Node:
    def __init__(self, name) -> None:
        self.name = name
        self.neighbors = []
        self.edges = []
    
    def __str__(self) -> str:
        return f"<{self.name}>"

class Edge:
    def __init__(self, v, w) -> None:
        self.nodes = [v, w]
        v.edges.append(self)
        v.neighbors.append(w)
        w.edges.append(self)
        w.neighbors.append(v)
    
    def __str__(self) -> str:
        return f"{self.nodes[0].name} - {self.nodes[1].name}"


In [3]:
def p1(inp):
    graph = Graph(inp)
    for n, node in graph.nodes.items():
        print(node)
        for m in node.neighbors:
            print(f"\t{m}")
        for e in node.edges:
            print(e)
        print()

    # for edge in graph.edges:
    #     print(edge)


In [4]:
p1(TEST)

<jqt>
	<rhn>
	<xhk>
	<nvd>
	<ntq>
jqt - rhn
jqt - xhk
jqt - nvd
ntq - jqt

<rhn>
	<jqt>
	<xhk>
	<bvb>
	<hfx>
jqt - rhn
rhn - xhk
rhn - bvb
rhn - hfx

<xhk>
	<jqt>
	<hfx>
	<rhn>
	<bvb>
	<ntq>
jqt - xhk
xhk - hfx
rhn - xhk
bvb - xhk
ntq - xhk

<nvd>
	<jqt>
	<cmg>
	<pzl>
	<qnr>
	<lhk>
jqt - nvd
cmg - nvd
pzl - nvd
qnr - nvd
nvd - lhk

<rsh>
	<frs>
	<pzl>
	<lsr>
	<rzs>
rsh - frs
rsh - pzl
rsh - lsr
rzs - rsh

<frs>
	<rsh>
	<qnr>
	<lhk>
	<lsr>
rsh - frs
frs - qnr
frs - lhk
frs - lsr

<pzl>
	<rsh>
	<lsr>
	<hfx>
	<nvd>
rsh - pzl
pzl - lsr
pzl - hfx
pzl - nvd

<lsr>
	<rsh>
	<pzl>
	<lhk>
	<rzs>
	<frs>
rsh - lsr
pzl - lsr
lsr - lhk
rzs - lsr
frs - lsr

<hfx>
	<xhk>
	<rhn>
	<bvb>
	<pzl>
	<ntq>
xhk - hfx
rhn - hfx
bvb - hfx
pzl - hfx
ntq - hfx

<cmg>
	<qnr>
	<nvd>
	<lhk>
	<bvb>
	<rzs>
cmg - qnr
cmg - nvd
cmg - lhk
cmg - bvb
rzs - cmg

<qnr>
	<cmg>
	<nvd>
	<rzs>
	<frs>
cmg - qnr
qnr - nvd
rzs - qnr
frs - qnr

<lhk>
	<cmg>
	<nvd>
	<lsr>
	<frs>
cmg - lhk
nvd - lhk
lsr - lhk
frs - lhk

<bvb>
	<cmg>
	<

In [5]:
# let's use nx
import networkx as nx 
import matplotlib.pyplot as plt 

def p1nx(inp, edges_to_remove=[]):
    G = nx.Graph()
    for line in inp:
        left, right = line.split(": ")
        for node in right.split(" "):
            G.add_edge(left.strip(), node.strip())

    for v,w in edges_to_remove:
        G.remove_edge(v, w)

    # this takes about a minute for the puzzle input so lets skip it for calculations
    # nx.draw(G, with_labels=True)
    # plt.show()

    conn_comps = [subg for subg in nx.connected_components(G)]
    return len(conn_comps[0]) * len(conn_comps[1])

In [6]:
p1nx(TEST, [["pzl","hfx"], ["nvd","jqt"], ["cmg","bvb"]])
# we can manually inspect this and see we want to remove pzl-hfx, nvd-jqt, cmg-bvb

54

In [7]:
# p1nx(A)
# manually inspect and see [["gbc","hxr"],["tmt","pnz"],["mvv", "xkz"]]
p1nx(A, [["gbc","hxr"],["tmt","pnz"],["mvv", "xkz"]])

569904